In [ ]:
# import statements
import random
import csv
import datetime as dt
import pymongo
import geohash2
import time

from time import sleep
from json import dumps
from kafka import KafkaProducer
from pymongo import MongoClient
from pprint import pprint

client = MongoClient()
db = client.fit5148_assignment_db
climate_fire = db.climate_fire

In [ ]:
AQUA_data = []

# Open the climate_historic CSV file 
with open('fire_AQUA_streaming.csv', 'r', encoding='utf-8-sig') as streaming_file: 
    # Skip header line
    next(streaming_file)
    
  # Read the fire_historic CSV file, append each row to the array
    for row in csv.reader(streaming_file):
        AQUA_data.append(row) 

In [ ]:
def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

if __name__ == '__main__':

    topic = 'AQUA' 
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    
    for i in range(len(AQUA_data)):
        values = random.choice(AQUA_data)
        geo3 = str(geohash2.encode(float(values[0]), float(values[1]), precision=3))
        data = topic + ', ' + dt.datetime.now().strftime("%X") + ', ' + str(values[0]) + ', ' + str(values[1]) + ', ' + str(values[2]) + ', ' + str(values[3]) + ', ' + str(values[4])
        
        publish_message(producer, topic, geo3, data)
        sleep(2)